# HiC benchmarking - SALSA2

* Advantages:
  * leverage assembly graph information for scaffolding
  * does not require user to specify chromosome number a priori. Run stops when the Hi-C information is exhausted
  * insensitive to assembly contiguity and Hi-C library variations
  * no need to tune of parameters for each dataset
  * better resolve unitig orientations, espetially short unitigs
  * show fewer orientation, ordering, and chimeric errors across a wide range of assembly contiguities. also robustness to different Hi-C libraries with varying intra-chromosomal contact frequencies. It generates more accturate scaffolds across all conditions tested compared to 3d-dna
  
* Cecilia's salsa1 pipeline:
  * BUSCO check on the PacBio assembly
  * HICUPS quality check on the HI-C data
  * mapping using bowtie2 and check mapping stats
  * clustering using salsa with raw reads
  * result was bad, checking for contamination using Kraken on the PacBio assembly

* comments from Cecilia:
> The Kraken result for MR is strange. Any contigs classified as bacteria or plant (by Kraken1 or Kraken2) were matched to Puccinia when blasted to NCBI. We decided to keep all sequences and ignore Kraken result.

> The latest MR version is at /input/genomic/fungal/Austropuccinia/psidii/PacBioAssembly/20190122 (MR_P2A.fasta, MR_H.fasta.gz and MR_artefacts.fasta) with P and H representing primary and haplotig alleles.

> check with Cecilia about double digestion input

## previous SALSA result

In [1]:
module load assemblathon_stats
module list

Currently Loaded Modulefiles:
  1) powerPlant/core                 8) geneid/v1.4.4
  2) texlive/20151117                9) wise/2.4.1
  3) pandoc/1.19.2                  10) hmmer/3.1b2
  4) git/2.21.0                     11) ncbi-blast/2.2.25
  5) perlbrew/0.76                  12) cegma/v2.5
  6) perl/5.28.0                    13) assemblathon_stats/2011_10_13
  7) asub/2.1


In [3]:
assemblathon_stats.pl /workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_ragoo_gapfilled_noContamination.ml10000.fasta

Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 283.
Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 408.

---------------- Information for assembly '/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_ragoo_gapfilled_noContamination.ml10000.fasta' ----------------


                                         Number of scaffolds        678
                                     Total size of scaffolds  292336213
                                            Longest scaffold   17261300
                                           Shortest scaffold      10017
                                 Number of scaffolds > 1K nt        678 100.0%
                                Number of scaffolds > 10K nt        678 100.0%
                               Number of scaffolds > 100K nt        259  38.2%
                                

## SALSA2

In [4]:
mkdir 015.salsa2

In [5]:
pwd

/powerplant/workspace/hraczw/github/GA/Gillenia_genome


In [6]:
workDir=/powerplant/workspace/hraczw/github/GA/Gillenia_genome/015.salsa2
GenomeDir=/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling
genome=scaff_ragoo_gapfilled_noContamination.ml10000.fasta
HiC_Dir=/workspace/hraczw/github/GA/Gillenia_genome/002.Fastp.trimming

### Read alignment and filtering

* mapping steps follow SALSA2 paper: using BWA (parameters: -t 12 -B 8)
[mapping details](https://github.com/ArimaGenomics/mapping_pipeline/blob/master/Arima_Mapping_UserGuide_A160146_v00.pdf)

In [7]:
module load bwa
module list

Currently Loaded Modulefiles:
  1) powerPlant/core                 8) geneid/v1.4.4
  2) texlive/20151117                9) wise/2.4.1
  3) pandoc/1.19.2                  10) hmmer/3.1b2
  4) git/2.21.0                     11) ncbi-blast/2.2.25
  5) perlbrew/0.76                  12) cegma/v2.5
  6) perl/5.28.0                    13) assemblathon_stats/2011_10_13
  7) asub/2.1                       14) bwa/0.7.17


#### Indexing genome for BWA mapping

In [14]:
bsub -J bwa_index \
-o $workDir/bwa_index.out \
-e $workDir/bwa_index.err \
"bwa index \
-p $workDir/ragoo_scaffolds_ml10000 \
$GenomeDir/$genome"

Job <273554> is submitted to default queue <lowpriority>.


#### BWA mapping

In [15]:
R1=$HiC_Dir/R1.cleaned.specifiedAdapter.short.Q15.fq.gz
R2=$HiC_Dir/R2.cleaned.specifiedAdapter.short.Q15.fq.gz

In [17]:
bjobs

No unfinished job found


: 255

In [13]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core                 9) wise/2.4.1
  2) texlive/20151117               10) hmmer/3.1b2
  3) pandoc/1.19.2                  11) ncbi-blast/2.2.25
  4) git/2.21.0                     12) cegma/v2.5
  5) perlbrew/0.76                  13) assemblathon_stats/2011_10_13
  6) perl/5.28.0                    14) bwa/0.7.17
  7) asub/2.1                       15) samtools/1.9
  8) geneid/v1.4.4


In [20]:
# adjust -B: panelty of mismatch

bsub -J bwa_R1 \
-n 60 \
-m wkoppb50 \
-o $workDir/bwa_R1.out \
-e $workDir/bwa_R1.err \
"bwa mem \
-t 60 \
-B 8 \
$workDir/ragoo_scaffolds_ml10000 \
$R1 | \
samtools view \
-Sb - > \
$workDir/R1.bam"

Job <276933> is submitted to default queue <lowpriority>.


In [19]:
bsub -J bwa_R2 \
-n 40 \
-o $workDir/bwa_R2.out \
-e $workDir/bwa_R2.err \
"bwa mem \
-t 40 \
-B 8 \
$workDir/ragoo_scaffolds_ml10000 \
$R2 | \
samtools view \
-Sb - > \
$workDir/R2.bam"

Job <273567> is submitted to default queue <lowpriority>.


In [21]:
bsub -J mappingStats \
-o $workDir/mappingStats_R1.out \
-e $workDir/mappingStats_R1.err \
"samtools flagstat \
$workDir/R1.bam \
> $workDir/R1_mappingStats.txt"

Job <276937> is submitted to default queue <lowpriority>.


In [26]:
cat $workDir/R1_mappingStats.txt

255955419 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
33362590 + 0 supplementary
0 + 0 duplicates
253432177 + 0 mapped (99.01% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [22]:
bsub -J mappingStats \
-o $workDir/mappingStats_R2.out \
-e $workDir/mappingStats_R2.err \
"samtools flagstat \
$workDir/R2.bam \
> $workDir/R2_mappingStats.txt"

Job <276938> is submitted to default queue <lowpriority>.


In [27]:
cat $workDir/R2_mappingStats.txt

255272077 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
32679248 + 0 supplementary
0 + 0 duplicates
251434016 + 0 mapped (98.50% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


#### Filtering "chimeric" reads
A single read jumping across ligation junction can be mapped with two ends to two different locations, each has high quality mapping score. only the 5'-side should be retained because the 3'-side can originate from the same contiguous DNA as the 5'-side of the reads mate-pair.

In [23]:
SCRIPTSDIR=/workspace/hraczw/github/programs/mapping_pipeline
FILTER=filter_five_end.pl

In [24]:
bsub -J samtools_R1 \
-o $workDir/filtered_R1.out \
-e $workDir/filtered_R1.err \
"samtools view \
-h \
$workDir/R1.bam | \
perl $SCRIPTSDIR/$FILTER | \
samtools view \
-Sb - > \
$workDir/R1_filtered.bam"

Job <276939> is submitted to default queue <lowpriority>.


In [25]:
bsub -J samtools_R2 \
-o $workDir/filtered_R2.out \
-e $workDir/filtered_R2.err \
"samtools view \
-h \
$workDir/R2.bam | \
perl $SCRIPTSDIR/$FILTER | \
samtools view \
-Sb - > \
$workDir/R2_filtered.bam"

Job <276940> is submitted to default queue <lowpriority>.


#### Combining read pairs & mapping quality filter

In [28]:
# faidx genome
COMBINER=$SCRIPTSDIR/two_read_bam_combiner.pl
FAIDX=$GenomeDir/$genome.fai

In [30]:
bsub -J combineReads \
-m wkoppb50 \
-o $workDir/combineReads.out \
-e $workDir/combineReads.err \
"perl $COMBINER \
$workDir/R1_filtered.bam \
$workDir/R2_filtered.bam \
samtools 10 | \
samtools view \
-bS \
-t $FAIDX - | \
samtools sort \
-o $workDir/pairs.bam -"

Job <276967> is submitted to default queue <lowpriority>.


In [31]:
# add read group
module load picard-tools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core                 9) wise/2.4.1
  2) texlive/20151117               10) hmmer/3.1b2
  3) pandoc/1.19.2                  11) ncbi-blast/2.2.25
  4) git/2.21.0                     12) cegma/v2.5
  5) perlbrew/0.76                  13) assemblathon_stats/2011_10_13
  6) perl/5.28.0                    14) bwa/0.7.17
  7) asub/2.1                       15) samtools/1.9
  8) geneid/v1.4.4                  16) picard-tools/2.9.4


In [32]:
bsub -J picard \
-m wkoppb50 \
-o $workDir/picard.out \
-e $workDir/picard.err \
"picard AddOrReplaceReadGroups \
INPUT=$workDir/pairs.bam \
OUTPUT=$workDir/pairs_updated.bam \
ID=Gillenia_HIC_clean \
LB=Gillenia_HIC_clean \
SM=GilleniaHiC \
PL=ILLUMINA \
PU=none"

Job <277014> is submitted to default queue <lowpriority>.


In [33]:
# mark duplicates

bsub -J markDuplicates \
-o $workDir/markDuplicates.out \
-e $workDir/markDuplicates.err \
"picard MarkDuplicates \
INPUT=$workDir/pairs_updated.bam \
OUTPUT=$workDir/pairs_updated_DuplicatesRemoved.bam \
METRICS_FILE=$workDir/duplicates_metrics.txt \
TMP_DIR=$workDir \
VALIDATION_STRINGENCY=LENIENT \
REMOVE_DUPLICATES=TRUE"

Job <277034> is submitted to default queue <lowpriority>.


In [34]:
bsub -J samtoolsIndex \
-o $workDir/samtoolsIndex.out \
-e $workDir/samtoolsIndex.err \
"samtools index \
$workDir/pairs_updated_DuplicatesRemoved.bam"

Job <282391> is submitted to default queue <lowpriority>.


In [41]:
bsub -J stats \
-o $workDir/stats.out \
-e $workDir/stats.err \
"perl $SCRIPTSDIR/get_stats.pl \
$workDir/pairs_updated_DuplicatesRemoved.bam \
> $workDir/pairs_updated_DuplicatesRemoved.bam.stats"

Job <282394> is submitted to default queue <lowpriority>.


In [42]:
cat $workDir/pairs_updated_DuplicatesRemoved.bam.stats

All            84749350
All intra      64591882
All intra 1kb  2786443
All intra 10kb 1903137
All intra 15kb 1772902
All intra 20kb 1682757
All inter      20157468


inter pairs are a lot less than those intra pairs.

### scaffolding

In [36]:
module load salsa
module list

Currently Loaded Modulefiles:
  1) powerPlant/core                10) hmmer/3.1b2
  2) texlive/20151117               11) ncbi-blast/2.2.25
  3) pandoc/1.19.2                  12) cegma/v2.5
  4) git/2.21.0                     13) assemblathon_stats/2011_10_13
  5) perlbrew/0.76                  14) bwa/0.7.17
  6) perl/5.28.0                    15) samtools/1.9
  7) asub/2.1                       16) picard-tools/2.9.4
  8) geneid/v1.4.4                  17) singularity/3
  9) wise/2.4.1                     18) salsa/v2.2


In [37]:
# prepare files for salsa
module load bedtools

In [38]:
bsub -J bamToBed \
-o $workDir/bamToBed.out \
-e $workDir/bamToBed.err \
"bamToBed -i \
$workDir/pairs_updated_DuplicatesRemoved.bam \
> $workDir/pairs_updated_DuplicatesRemoved.bed"

Job <282393> is submitted to default queue <lowpriority>.


In [43]:
bsub -J bedsort \
-o $workDir/bedsort.out \
-e $workDir/bedsort.err \
"sort -k 4 $workDir/pairs_updated_DuplicatesRemoved.bed \
> tmp && mv tmp $workDir/pairs_updated_DuplicatesRemoved.bed"

Job <282408> is submitted to default queue <lowpriority>.


In [44]:
bjobs

JOBID   USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
282408  hraczw  RUN   lowpriorit aklppj31    aklppb47    bedsort    Sep  2 17:39


In [45]:
# generate contig index
samtools faidx $GenomeDir/$genome

In [46]:
faiFile=$GenomeDir/$genome.fai

In [47]:
mkdir $workDir/salsa2

In [50]:
run_pipeline.py -h

usage: run_pipeline.py [-h] -a ASSEMBLY -l LENGTH -b BED [-o OUTPUT]
                       [-c CUTOFF] [-g GFA] [-e ENZYME] [-i ITER] [-x DUP]
                       [-s EXP] [-m CLEAN] [-f FILTER] [-p PRNT]

SALSA Iterative Pipeline

optional arguments:
  -h, --help            show this help message and exit
  -a ASSEMBLY, --assembly ASSEMBLY
                        Path to initial assembly
  -l LENGTH, --length LENGTH
                        Length of contigs at start
  -b BED, --bed BED     Bed file of alignments sorted by read names
  -o OUTPUT, --output OUTPUT
                        Output directory to put results
  -c CUTOFF, --cutoff CUTOFF
                        Minimum contig length to scaffold, default=1000
  -g GFA, --gfa GFA     GFA file for assembly
  -e ENZYME, --enzyme ENZYME
                        Restriction Enzyme used for experiment
  -i ITER, --iter ITER  Number of iterations to run, default = 3
  -x DUP, --dup DUP     File containing duplicated contig informati

In [56]:
python /workspace/hraczw/github/programs/juicebox_scripts/juicebox_scripts/makeAgpFromFasta.py \
$GenomeDir/$genome $GenomeDir/$genome.agp

In [59]:
# run salsa scaffolding, setting is the same with Cecilia's salsa1 in order to compare the results

bsub -J salsa \
-o $workDir/salsa2_ml10k_i5_glen/salsa2.out \
-e $workDir/salsa2_ml10k_i5_glen/salsa2.err \
"run_pipeline.py \
-a $GenomeDir/$genome \
-l $faiFile \
-b $workDir/pairs_updated_DuplicatesRemoved.bed \
-c 10000 \
-e GATC \
-i 5 \
-s 300000000 \
-o $workDir/salsa2_ml10k_i5_glen/Gillenia_ragoo.Scaffolds \
-m yes"

Job <282435> is submitted to default queue <lowpriority>.


In [57]:
ls $GenomeDir

10x_S1-S3_R1.fastq.gz
busco_ragoo_non-haplotigs.err
busco_ragoo_non-haplotigs.out
fq-to-fa.minion.err
fq-to-fa.minion.out
fq-to-fa.p.err
fq-to-fa.p.out
G3-2-AK3717_S1_L001_R1_001.fastq.gz
G3-2-AK3717_S1_L001_R1_001_preseq.err
G3-2-AK3717_S1_L001_R1_001_preseq.out
G3-2-AK3718_S2_L001_R1_001.fastq.gz
G3-2-AK3718_S2_L001_R1_001_preseq.err
G3-2-AK3718_S2_L001_R1_001_preseq.out
G3-2-AK3719_S3_L001_R1_001.fastq.gz
G3-2-AK3719_S3_L001_R1_001_preseq.err
G3-2-AK3719_S3_L001_R1_001_preseq.out
G3-2-AK3720_S4_L001_R1_001.fastq.gz
G3-2-AK3720_S4_L001_R1_001_preseq.err
G3-2-AK3720_S4_L001_R1_001_preseq.out
gapcloser.err
gapcloser_links.err
gapcloser_links_i8.err
gapcloser_links_i8.out
gapcloser_links.out
gapcloser.out
gmcloser_filled-1m.nucmer.delta
gmcloser_filled-bowtie_align
gmcloser_filled-score.txt
gmcloser_filled-subcon.fa
gmcloser_filled.submit.log
merge_10x.err
merge_10x.out
ragoo_non-haplotigs_BUSCO_tmp
readFiles_R1.txt
readFiles.txt
run_ragoo_non-haplotigs_BUSCO
scaff_links_i10_gapFilled.c

In [51]:
module load assemblathon_stats

In [52]:
# salsa2 assembly without recommended mapping pipeline
assemblathon_stats.pl $workDir/salsa2/Gillenia_ragoo.Scaffolds/scaffolds_FINAL.fasta

Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 283.
Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 408.

---------------- Information for assembly '/powerplant/workspace/hraczw/github/GA/Gillenia_genome/015.salsa2/salsa2/Gillenia_ragoo.Scaffolds/scaffolds_FINAL.fasta' ----------------


                                         Number of scaffolds        669
                                     Total size of scaffolds  292351713
                                            Longest scaffold   17261300
                                           Shortest scaffold      10094
                                 Number of scaffolds > 1K nt        669 100.0%
                                Number of scaffolds > 10K nt        669 100.0%
                               Number of scaffolds > 100K nt        276  41.3%
                       

In [53]:
# salsa2 assembly stats
assemblathon_stats.pl $GenomeDir/$genome

Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 283.
Use of qw(...) as parentheses is deprecated at /software/bioinformatics/assemblathon_stats-2011_10_13/assemblathon_stats.pl line 408.

---------------- Information for assembly '/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/scaff_ragoo_gapfilled_noContamination.ml10000.fasta' ----------------


                                         Number of scaffolds        678
                                     Total size of scaffolds  292336213
                                            Longest scaffold   17261300
                                           Shortest scaffold      10017
                                 Number of scaffolds > 1K nt        678 100.0%
                                Number of scaffolds > 10K nt        678 100.0%
                               Number of scaffolds > 100K nt        259  38.2%
                                